In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

In [7]:

# Replace with the path to your exported Apple Music XML file
XML_FILE_PATH = "../data/raw/Library.xml"

# Parse the XML file
tree = ET.parse(XML_FILE_PATH)
root = tree.getroot()

In [87]:
songs = []
# Find all Replay Playlists and add track ids to list
for child in root.findall('dict/dict/dict'):
    track_info = {}
    elements = list(child)
    for i in range(len(elements) - 1):
        if elements[i].tag == "key":
            track_info[elements[i].text] = elements[i + 1].text
        if "Name" in track_info:
            songs.append({
                "Track ID": track_info.get("Track ID", "N/A"),
                "Title": track_info.get("Name", "N/A"),
                "Artist": track_info.get("Artist", "N/A"),
                "Composer": track_info.get("Composer", "N/A"),
                "Album": track_info.get("Album", "N/A"),
                "Genre": track_info.get("Genre", "N/A"),
                "Release Date": track_info.get("Release Date", "N/A"),
                "Kind": track_info.get("Kind", "N/A"),
                "Total Time (ms)": track_info.get("Total Time", "N/A"),
                "Date Modified": track_info.get("Date Modified", "N/A"),
                "Date Added": track_info.get("Date Added", "N/A")
            })
        
    
            
print(len(songs))

103091


In [100]:
songs_df = pd.DataFrame(songs)
print(songs_df.info())
# Identify "N/A" or Empty Strings
text_cols = ["Track ID", "Title", "Artist", "Composer", "Album", "Genre", "Kind", "Total Time (ms)"]
for col in text_cols:
    print(f"{col} - 'N/A' count:", (songs_df[col] == "N/A").sum())

# Convert "N/A" & Empty Strings to Null
songs_df.replace("N/A", pd.NA, inplace=True)

# Recheck "N/A" or Empty Strings
text_cols = ["Track ID", "Title", "Kind", "Total Time (ms)"]
for col in text_cols:
    print(f"{col} - 'N/A' count:", (songs_df[col] == "N/A").sum())

# Convert date columns to datetime format
date_cols = ["Release Date", "Date Modified", "Date Added"]
for col in date_cols:
    songs_df[col + " (Raw)"] = songs_df[col]  # Keep the raw values
    songs_df[col] = pd.to_datetime(songs_df[col], format="%Y-%m-%dT%H:%M:%SZ", errors="coerce")

# Sort by "Date Modified" (newest first) and drop duplicate "Track ID" entries
songs_df = songs_df.sort_values(by="Date Modified", ascending=False).drop_duplicates(subset="Track ID")

songs_df.reset_index()
print(songs_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103091 entries, 0 to 103090
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Track ID         103091 non-null  object
 1   Title            103091 non-null  object
 2   Artist           103091 non-null  object
 3   Composer         103091 non-null  object
 4   Album            103091 non-null  object
 5   Genre            103091 non-null  object
 6   Release Date     103091 non-null  object
 7   Kind             103091 non-null  object
 8   Total Time (ms)  103091 non-null  object
 9   Date Modified    103091 non-null  object
 10  Date Added       103091 non-null  object
dtypes: object(11)
memory usage: 8.7+ MB
None
Track ID - 'N/A' count: 0
Title - 'N/A' count: 0
Artist - 'N/A' count: 3992
Composer - 'N/A' count: 17468
Album - 'N/A' count: 15735
Genre - 'N/A' count: 20100
Kind - 'N/A' count: 23998
Total Time (ms) - 'N/A' count: 30608
Track ID - 'N/A' count

In [91]:
songs_df = songs_df.drop(
songs_df = songs_df.reset_index(drop = True)
songs_df.head()

,Track ID,Title,Artist,Composer,Album,Genre,Release Date,Kind,Total Time (ms),Date Modified,Date Added,Release Date (Raw),Date Modified (Raw),Date Added (Raw)
0,5253,Wanna Know,Mya Lanae,"Ethan Smith Jr., Abel Herrera & Mya Smith",Wanna Know - Single,R&B/Soul,2022-03-18 12:00:00,Apple Music AAC audio file,178573,2025-03-16 18:40:25,NaT,2022-03-18T12:00:00Z,2025-03-16T18:40:25Z,N/A
1,5251,Anxiety,Megan Thee Stallion,Kevin Price & Megan Pete,Traumazine,Hip-Hop/Rap,2022-08-12 07:00:00,Apple Music AAC audio file,135867,2025-03-16 18:32:18,NaT,2022-08-12T07:00:00Z,2025-03-16T18:32:18Z,N/A
2,5171,Villain Era,Peachkka,Nikkole Maresa Baker & Kerby Joseph,Villain Era - Single,Hip-Hop/Rap,2022-03-05 12:00:00,Apple Music AAC audio file,150079,2025-03-10 16:29:07,2025-03-10 16:29:07,2022-03-05T12:00:00Z,2025-03-10T16:29:07Z,2025-03-10T16:29:07Z
3,5153,Innerstood,Shariya Wise,Ashlee Shariya Lombard,Baddie Affirmations,Hip-Hop/Rap,2024-11-22 12:00:00,Apple Music AAC audio file,147879,2025-03-10 15:50:16,2025-03-10 15:50:16,2024-11-22T12:00:00Z,2025-03-10T15:50:16Z,2025-03-10T15:50:16Z
4,5157,Sweet Like Candyland,Shariya Wise,Ashlee Shariya Lombard,Baddie Affirmations,Hip-Hop/Rap,2024-11-22 12:00:00,Apple Music AAC audio file,187826,2025-03-10 15:50:16,2025-03-10 15:50:16,2024-11-22T12:00:00Z,2025-03-10T15:50:16Z,2025-03-10T15:50:16Z
